<a href="https://colab.research.google.com/github/Antonsen2/wildfire-ai/blob/research/optimize_model_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.8 MB/s eta 0:00:00


In [2]:
import keras_tuner
import os.path
import itertools
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from tensorflow import keras
from pathlib import Path
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image

from tensorflow.keras import layers, models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

--2023-01-16 05:08:25--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-16 05:08:25 (69.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Load dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
DATA_PATH = '/content/drive/MyDrive/full'
IMG_DIR = Path(DATA_PATH)

In [6]:
walk_through_dir(DATA_PATH)

There are 3 directories and 0 images in '/content/drive/MyDrive/full'.
There are 0 directories and 2071 images in '/content/drive/MyDrive/full/fire'.
There are 0 directories and 2150 images in '/content/drive/MyDrive/full/satellite'.
There are 0 directories and 500 images in '/content/drive/MyDrive/full/nonfire'.


## Prepare Dataset

In [7]:
def split_df(df: pd.DataFrame, cols: list, **kwargs) -> pd.DataFrame:
    df1 = pd.DataFrame()

    for k, v in kwargs.items():
        df2 = df.loc[df['Label'] == k][cols].sample(n=v)
        df1 = pd.concat([df1, df2])

    df1.index = range(len(df1))
    return df1

In [8]:
filepaths = list(IMG_DIR.glob(r'**/*.JPG')) + list(IMG_DIR.glob(r'**/*.jpg')) + list(IMG_DIR.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)
image_df

,Filepath,Label
0,/content/drive/MyDrive/full/fire/fire-837.jpg,fire
1,/content/drive/MyDrive/full/fire/fire-1095.jpg,fire
2,/content/drive/MyDrive/full/fire/fire-759.jpg,fire
3,/content/drive/MyDrive/full/fire/fire-1047.jpg,fire
4,/content/drive/MyDrive/full/fire/fire-821.jpg,fire
...,...,...
4716,/content/drive/MyDrive/full/nonfire/forest-396...,nonfire
4717,/content/drive/MyDrive/full/nonfire/forest-401...,nonfire
4718,/content/drive/MyDrive/full/nonfire/forest-381...,nonfire
4719,/content/drive/MyDrive/full/nonfire/forest-458...,nonfire


In [9]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [10]:
df_full = split_df(
    df=image_df.copy(),
    cols=['Filepath', 'Label'],
    fire=2071,
    nonfire=500
)

In [11]:
train_df, test_df = train_test_split(
    df_full, test_size=0.2, shuffle=True, random_state=42
)

In [12]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
RANDOM_SEED = 42

In [13]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMG_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=RANDOM_SEED,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMG_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=RANDOM_SEED,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMG_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 1645 validated image filenames belonging to 2 classes.
Found 411 validated image filenames belonging to 2 classes.
Found 515 validated image filenames belonging to 2 classes.


# Model Optimization

In [30]:
def build_model(hp):
    resize_and_rescale = tf.keras.Sequential([
      layers.experimental.preprocessing.Resizing(*IMG_SIZE),
      layers.experimental.preprocessing.Rescaling(1./255),])

    pretrained_model = MobileNetV2(
      input_shape=IMG_SIZE+ (3,),
      include_top=False,
      weights='imagenet',
      pooling='avg')

    pretrained_model.trainable = False
    inputs = pretrained_model.input
    X = resize_and_rescale(inputs)

    X = Dense(
        units=hp.Int("units", min_value=32, max_value=512, step=32),
        activation=hp.Choice("activation1", ["relu", "tanh", "sigmoid"], ordered=False))(pretrained_model.output)
    X = Dropout(0.2)(X)
    X = Dense(
        units=hp.Int("units", min_value=32, max_value=512, step=32),
        activation=hp.Choice("activation2", ["relu", "tanh", "sigmoid"], ordered=False))(X)
    X = Dropout(0.2)(X)


    outputs = Dense(2, activation=hp.Choice("activation3", ["softmax", "sigmoid", "linear"], ordered=False))(X)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=Adam(hp.Choice('learning_rate',
                                 [1e-2, 1e-4], ordered=False)),
                                 loss = 'categorical_crossentropy',
                                 metrics = ['accuracy'])

    return model

## Start the search

In [31]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="keras_tuning",
    project_name="wildfire",
)

In [32]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
activation2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
activation3 (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmoid', 'linear'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.0001], 'ordered': False}


In [33]:
tuner.search(
        train_images,
        steps_per_epoch=len(train_images),
        validation_data=val_images,
        validation_steps=len(val_images),
        epochs=3
    )

Trial 3 Complete [00h 00m 53s]
val_accuracy: 0.09732360183261335

Best val_accuracy So Far: 0.9939172863960266
Total elapsed time: 00h 02m 41s


## Query the results

In [34]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 block_4_depthwise_relu (ReLU)  (None, 28, 28, 192)  0           ['block_4_depthwise_BN[0][0]']   
                                                                                                  


 block_4_project (Conv2D)       (None, 28, 28, 32)   6144        ['block_4_depthwise_relu[0][0]'] 


 block_4_project_BN (BatchNorma  (None, 28, 28, 32)  128         ['block_4_project[0][0]']        


 lization)                                                                                        
                                                                                                  
 block_4_add (Add)              (None, 28, 28, 32)   0           ['block_3_project_BN[0][0]',     
                                                                  'block_4_project_BN[0][0]']     


                                                                                                  
 block_5_expand (Conv2D)        (None, 28, 28, 192)  6144        ['block_4_add[0][0]']            


 block_5_expand_BN (BatchNormal  (None, 28, 28, 192)  768        ['block_5_expand[0][0]']         


 ization)                                                                                         


 block_5_expand_relu (ReLU)     (None, 28, 28, 192)  0           ['block_5_expand_BN[0][0]']      
                                                                                                  
 block_5_depthwise (DepthwiseCo  (None, 28, 28, 192)  1728       ['block_5_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_5_depthwise_BN (BatchNor  (None, 28, 28, 192)  768        ['block_5_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_5_depthwise_relu (ReLU)  (None, 28, 28, 192)  0           ['block_5_depthwise_BN[0][0]']   
                                                                                                  
 block_5_p

In [35]:
tuner.results_summary()

Results summary
Results in keras_tuning/wildfire
Showing 10 best trials
Trial summary
Hyperparameters:
units: 512
activation1: tanh
activation2: tanh
activation3: sigmoid
learning_rate: 0.0001
Score: 0.9939172863960266
Trial summary
Hyperparameters:
units: 416
activation1: sigmoid
activation2: relu
activation3: linear
learning_rate: 0.0001
Score: 0.8965936601161957
Trial summary
Hyperparameters:
units: 256
activation1: tanh
activation2: sigmoid
activation3: linear
learning_rate: 0.01
Score: 0.09732360183261335


In [40]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters()[0]

print(best_hps.get("activation1"))
print(best_hps.get("activation2"))
print(best_hps.get("activation3"))
print(best_hps.get("units"))
print(best_hps.get("learning_rate"))

tanh
tanh
sigmoid
512
0.0001
